In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  
import pandas as pd
import time
import requests
import credentials
from datetime import date, datetime

In [2]:
def accept_cookies_if_asked(driver):
    try:
        driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
    except:
        pass

In [3]:
class Data:
    def __init__(self):
        self.ids = []
        self.prices = []
        self.address = []
        self.postcodes = []
        self.latitudes = []
        self.longitudes = []

    def add_page_data(self, driver):
        properties = driver.find_elements(By.CLASS_NAME, "l-searchResult")
        for property in properties:
            self.ids.append(property.find_element(By.CLASS_NAME, "propertyCard-anchor").get_attribute('id'))
            self.prices.append(property.find_element(By.CLASS_NAME, "propertyCard-priceValue").text)
            self.address.append(property.find_element(By.CLASS_NAME, "propertyCard-address").get_attribute('title'))

    def transform_location_data(self):
        self.postcodes = []
        self.latitudes = []
        self.longitudes = []
        for addy in self.address:
            addy = addy + ", London, UK"
            addy = addy.replace(' ', '%20')
            r = requests.get(rf'https://dev.virtualearth.net/REST/v1/Locations?q={addy}&key={credentials.bingmaps_api_key}')
            if r.status_code >= 300:
                print(r.status_code)
                raise Exception(r.reason)
            try:
                self.postcodes.append(r.json()['resourceSets'][0]['resources'][0]['address']['postalCode'])
            except KeyError:
                self.postcodes.append(None)
            self.latitudes.append(r.json()['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates'][0])
            self.longitudes.append(r.json()['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates'][1])

    def transform_prices(self):
        self.prices = [int(price[1:-4].replace(',','')) for price in self.prices]

    def make_df(self):
        return pd.DataFrame({'id': self.ids, 'price': self.prices, 'address': self.address, 
                             'postcode': self.postcodes, 'latitude': self.latitudes, 'longitude': self.longitudes})

    def zip_property(self):
        ''''This method zips up all of the attributes needed to 
        insert into the property table in the database'''
        return list(zip([int(id.replace('prop','')) for id in self.ids], self.address, self.postcodes, [round(lat, 8) for lat in self.latitudes], [round(lon, 8) for lon in self.longitudes]))
    
    def zip_price(self, date_today):
        ''''This method zips up all of the attributes needed to 
        insert into the price table in the database'''
        return [(int(id.replace('prop','')), str(date_today), self.prices[i]) for i, id in enumerate(self.ids)]


In [4]:
date_today = date.today()
url = r'''https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E92829&maxBedrooms=2&minBedrooms=2&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='''
driver = webdriver.Chrome()
driver.get(url)

time.sleep(2)
accept_cookies_if_asked(driver)

data = Data()
data.add_page_data(driver)
pages_scanned = 1

button = driver.find_element(By.XPATH, "//button[@title='Next page']")

while button.is_enabled():
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(button))
    button.click()
    accept_cookies_if_asked(driver)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'propertyCard-anchor')))
    data.add_page_data(driver)
    pages_scanned += 1
    button = driver.find_element(By.XPATH, "//button[@title='Next page']")

driver.close()


In [7]:
len(data.prices)

1050

In [3]:
addy = "Kuala Gardens, London, SW16 3LP" + ", London, UK"
addy = addy.replace(' ', '%20')
r = requests.get(rf'https://maps.googleapis.com/maps/api/geocode/json?address={addy}&key={credentials.googlemaps_api_key}')
r.json()['results']

[{'address_components': [{'long_name': 'Kuala Gardens',
    'short_name': 'Kuala Gardens',
    'types': ['route']},
   {'long_name': 'London', 'short_name': 'London', 'types': ['postal_town']},
   {'long_name': 'Greater London',
    'short_name': 'Greater London',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'England',
    'short_name': 'England',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United Kingdom',
    'short_name': 'GB',
    'types': ['country', 'political']},
   {'long_name': 'SW16 3LP',
    'short_name': 'SW16 3LP',
    'types': ['postal_code']}],
  'formatted_address': 'Kuala Gardens, London SW16 3LP, UK',
  'geometry': {'bounds': {'northeast': {'lat': 51.41418700000003,
     'lng': -0.1203358},
    'southwest': {'lat': 51.41365689999996, 'lng': -0.1217817}},
   'location': {'lat': 51.413665, 'lng': -0.121013},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': 51.41527093029149,
  

In [31]:
r.json()['results'][0]
result = r.json()['results']
for comp in result[0]['address_components']:
    if "postal_code" in comp['types'][0]:
        print(comp["long_name"])
lat = result[0]['geometry']['location']['lat']
lng = result[0]['geometry']['location']['lng']
print(lat)
print(lng)

52.9540223
-1.1549892


In [6]:
data.transform_prices()
data.transform_location_data()

In [14]:
import pyodbc

In [17]:
def load_data(conn, query, data):
    cursor = conn.cursor()
    cursor.executemany(query, data)
    cursor.commit()
    cursor.close()

In [15]:
endpoint = credentials.db_endpoint
conn = pyodbc.connect('Driver={SQL Server};'
                  'Server='+credentials.db_endpoint+';'
                  'Database='+credentials.db_database+';'
                  'UID='+credentials.db_user+';'
                  'PWD='+credentials.db_password+';'
                  'Trusted_Connection=no;')

In [ ]:
property_tuples = data.zip_property()
property_query = '''
                WITH source AS (
                SELECT * FROM (VALUES (?, ?, ?, ?, ?)) s(prop_id, address, postcode, latitude, longitude)
                )
                INSERT INTO dbo.property
                (prop_id, address, postcode, latitude, longitude)
                SELECT source.prop_id, source.address, source.postcode, 
                source.latitude, source.longitude 
                FROM source
                WHERE NOT EXISTS
                (SELECT 1 FROM dbo.property target
                WHERE target.prop_id = source.prop_id)
                '''
load_data(conn, property_query, property_tuples)

price_tuples = data.zip_price(date_today=date_today)
price_query = '''
                WITH source AS (
                SELECT * FROM (VALUES (?, ?, ?)) s(prop_id, date, price)
                )
                INSERT INTO dbo.price
                (prop_id, date, price)
                SELECT source.prop_id, source.date, source.price 
                FROM source
                WHERE NOT EXISTS
                (SELECT 1 FROM dbo.price target
                WHERE target.prop_id = source.prop_id
                AND target.date = source.date)
                '''

load_data(conn, price_query, price_tuples)

conn.close()

In [ ]:
def access_url(url):
    headers = {
        'User-Agent': 'Mozilla'
    }
    s = requests.Session()
    s.headers.update(headers)
    r = s.get(url)
    if r.status_code >= 300:
        print(r.status_code)
        raise Exception(r.reason)

def get_data_from_csv(url):
    return pd.read_csv(url)

In [257]:
cursor = conn.cursor()
query = '''select MAX(date) from dbo.interest_rates'''
cursor.execute(query)
results = cursor.fetchall()

In [258]:
previous_date = datetime.strptime(results[0][0], "%Y-%m-%d")
prev_month = previous_date.strftime("%b")
prev_day = f"{previous_date.day:02d}"
prev_year = previous_date.year
curr_month = date_today.strftime("%b")
curr_day = f"{date_today.day:02d}"
curr_year = date_today.year

url = rf"https://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes&Datefrom={prev_day}%2F{prev_month}%2F{prev_year}&Dateto={curr_day}%2F{curr_month}%2F{curr_year}&SeriesCodes=IUMSOIA&CSVF=TN&UsingCodes=Y&VPD=Y&VFD=N"
access_url(url)
get_data_from_csv(url)
interest_rates = get_data_from_csv(url)

In [259]:
interest_rates

,DATE,IUMSOIA
0,30 Jun 2023,4.9286


In [123]:
interest_rate_tuples = list(zip(interest_rates.DATE, interest_rates.IUMSOIA))
interest_rate_query = '''
                        WITH source AS (
                        SELECT * FROM (VALUES (?, ?)) s(date, rate)
                        )
                        INSERT INTO dbo.interest_rates
                        (date, IUMSOIA)
                        SELECT source.date, source.rate
                        FROM source
                        WHERE NOT EXISTS
                        (SELECT 1 FROM dbo.interest_rates target
                        WHERE target.date = source.date
                        )
                        '''
load_data(conn, interest_rate_query, interest_rate_tuples)

In [18]:
import yfinance as yf

ticker = yf.Ticker("SPY")
sp500 = ticker.history(period="5d")
sp500_now = sp500[sp500.index == max(sp500.index)]

SPY_tuple = [(date_time.strftime("%Y-%m-%d"), close) for (date_time, close) in zip(sp500_now.index.date, round(sp500_now['Close'], 5))]

SPY_query = '''
            WITH source AS (
            SELECT * FROM (VALUES (?, ?)) s(date, [close])
            )
            INSERT INTO dbo.SPY_price
            (date, [close])
            SELECT source.date, source.[close]
            FROM source
            WHERE NOT EXISTS
            (SELECT 1 FROM dbo.SPY_price target
            WHERE target.date = source.date
            )
            '''
load_data(conn, SPY_query, SPY_tuple)

In [8]:
endpoint = credentials.db_endpoint
conn = pyodbc.connect('Driver={SQL Server};'
                  'Server='+credentials.db_endpoint+';'
                  'Database='+credentials.db_database+';'
                  'UID='+credentials.db_user+';'
                  'PWD='+credentials.db_password+';'
                  'Trusted_Connection=no;')

# load to property table
property_tuples = data.zip_property()
cursor = conn.cursor()
cursor.executemany('''
                    WITH source AS (
                    SELECT * FROM (VALUES (?, ?, ?, ?, ?)) s(prop_id, address, postcode, latitude, longitude)
                    )
                    INSERT INTO dbo.property
                    (prop_id, address, postcode, latitude, longitude)
                    SELECT source.prop_id, source.address, source.postcode, 
                    source.latitude, source.longitude 
                    FROM source
                    WHERE NOT EXISTS
                    (SELECT 1 FROM dbo.property target
                    WHERE target.prop_id = source.prop_id)
                   ''', property_tuples)
cursor.commit()
cursor.close()

In [22]:
# load to price table
price_tuples = data.zip_price(date_today=date_today)
cursor = conn.cursor()
cursor.executemany('''
                    WITH source AS (
                    SELECT * FROM (VALUES (?, ?, ?)) s(prop_id, date, price)
                    )
                    INSERT INTO dbo.price
                    (prop_id, date, price)
                    SELECT source.prop_id, source.date, source.price 
                    FROM source
                    WHERE NOT EXISTS
                    (SELECT 1 FROM dbo.price target
                    WHERE target.prop_id = source.prop_id
                    AND target.date = source.date)
                   ''', price_tuples)
cursor.commit()
cursor.close()

conn.close()

In [120]:
list(zip(interest_rates.DATE, interest_rates.IUMSOIA))

[('31 Jul 2022', 1.1909),
 ('31 Aug 2022', 1.6896),
 ('30 Sep 2022', 2.1901),
 ('31 Oct 2022', 2.1838),
 ('30 Nov 2022', 2.9273),
 ('31 Dec 2022', 3.4282),
 ('31 Jan 2023', 3.4269),
 ('28 Feb 2023', 3.927),
 ('31 Mar 2023', 4.1777),
 ('30 Apr 2023', 4.1792),
 ('31 May 2023', 4.4278),
 ('30 Jun 2023', 4.9286)]